# Note using DESeq in a consistant way
All of our DESeq work was not using R Notebooks. See extraCellularRNA/R/notebooks/ and extraCellularRNA/R/notebooks/reverseEngineerDESeq/ . We can not use DESeq and our current workflow to process the GTEx and TCGA data. These data sets are to big

Part of our work around is run DESeq on a count matrix instead of using tximport to read roughly 29K quant.sf files.

## Reverse engineer how DESeq works with salmon quant files
additional references: extraCellularRNA/juypterNotebooks/spark

extraCellularRNA/R/notebooks/kras.ipsc.DESeq.normalize.v2.nb.html

```
'KRAS.IPSC.day.5.gene'=list(
    'rootDir' = krasDataRoot,
    'findFilesFunc' = findDay5KrasIpscDataFilesGencode_v35_ucsc_rmsk_salmon_v1_3.0, 
    # colData is the meta data data frame
    'getColDFFunc' = getKrasIPSCDay5ColDataDF,        
    'tx2MappingFile'='gencode.v35.ucsc.rmsk.tx.to.gene.csv',
    'tx2MappingDir' = file.path('/home/kimlab/genomes.annotations/gencode.35'),
    'mapToIdName'='gene',
    'normalizedFileName'='kras.ipsc.day.5.DESeq.normalize.v2',
    'outputRoot' = "/home/rstudio/extraCellularRNA/data/R/output",
    #'outputRoot' = "/home/aedavids/extraCellularRNA/data/R/output",
    #'outputRoot' = "/home/kimlab/aedavids/extraCellularRNA/data/R/output",
    'transcriptMapperDFFunc'= getTEGeneMapper,
    # ignoreAfterBar logical, whether to split the tx id on the '|' character to 
    # facilitate matching with the tx id in tx2gene (default FALSE)
    #'ignoreAfterBar'=FALSE #bug we do not get gene names in output
    'ignoreAfterBar'=FALSE,
    'txOut'=FALSE,
    
    # getColDFFunc returns a data frame with columns
    #   factor sampleType ['bulk', 'exo']
    #   factor treatment  ['ctrl', 'kras']
    #   factor day        [5]
    'designStr'="~ sampleType + treatment"
    ),
```

# <span style="color:red">main runNomalization()</span>

```
runNomalization <- function(parametersList, dataSetNamesList) {
    #parameters <- parametersList$TE.gene
    for (dataSetName in dataSetNamesList) {
      parameters <- parametersList[[dataSetName]]
      error = missingParmeters(dataSetName, parameters )
      
      if (error ) {
        next # ie continue in most languages
      }

      retList    <- loadAndNormlize(parameters)
      dsCountsDF <- retList$ddsCountsDF
      dds        <- retList$dds
      fileList   <- retList$fileList
      
      dsCountsDF <- fixColNames(fileList, dsCountsDF)
      saveNormalizedDF(dsCountsDF, parameters)
      
      retList    <- runDifferentialExpression(dds, parameters)
      coefList <- retList[ "coefList" ]
      unShrunkResults <- retList[ "unShrunkResults" ]
      lfcShrinkResults <- retList[ "lfcShrinkResults" ]

      # save results
      # warning(paste("\nAEDWIP runNomalization coefList:", coefList))
      # warning(paste("\nAEDWIP runNomalization coefList[1]:", coefList[1]))
      # cat(paste("\n ???? [[1]]", coefList[[1]]))
      # # cat(paste("\n ???? wtf", coefList[2]))
      # cat(paste("\n ???? [[2]]", coefList[[2]]))
      # cat(paste("\n ???? 1 1", coefList[1][1]))
      # cat(paste("\n ???? 1 2", coefList[1][2]))
      # warning(paste("\nAEDWIP runNomalization coefList[2]:", coefList[2]))
      
      # coefList$coefList12]
      # [1] "Intercept"
      # coefList$coefList[2]
      # [1] "diseaseState_PDAC_vs_healthy"
      
      nameAEDWIP = paste0(coefList$coefList[2], ".unShrunk")
      saveResults( unShrunkResults, nameAEDWIP, parameters )
      nameAEDWIP = paste0(coefList$coefList[2], ".lfcShrink")
      saveResults( lfcShrinkResults, nameAEDWIP, parameters )
      
  }
}
```

## How does our mapping file deal with T.E.?

```
tx2MappingFile <- '/private/groups/kimlab/genomes.annotations/gencode.35/gencode.v35.ucsc.rmsk.tx.to.gene.csv'
```

<b>It is hard to see that this is a 2 column csv file, txtId, geneId</b>
```
(base) [aedavids@mustard gencode.35]$ head gencode.v35.ucsc.rmsk.tx.to.gene.csv 
ENST00000456328.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000362751.1|DDX11L1-202|DDX11L1|1657|processed_transcript|,DDX11L1
ENST00000450305.2|ENSG00000223972.5|OTTHUMG00000000961.2|OTTHUMT00000002844.2|DDX11L1-201|DDX11L1|632|transcribed_unprocessed_pseudogene|,DDX11L1
```


```
tail gencode.v35.ucsc.rmsk.tx.to.gene.csv
hg38_rmsk_L1PA3_range=chr22_KI270739v1_random:49092-50070_5'pad=0_3'pad=0_strand=+_repeatMasking=none,L1PA3
hg38_rmsk_ALR/Alpha_range=chr22_KI270739v1_random:50071-73985_5'pad=0_3'pad=0_strand=-_repeatMasking=none,ALR/Alpha
```

# <span style="color:red"> runNomalization -> loadAndNormlize </span>
DESeqUtils.R

0. <span style="color:red"> state of our dds object, (DESeqDataSet), after running is </span>
1. count matrix is set
2. counts are grouped by gene id and counted
3. counts have been converted to integers
4. sanity check error if count < 0
5. prefilter counts
6. estimateSizeFactors have been calculated 

```
loadAndNormlize <- function(parameters) {

    ...
    
 ddsList <- getDESeqDataSet(rootDir=parameters$rootDir,
                             tx2MappingFilePath,
                             findFilesFunc= parameters$findFilesFunc, 
                             getColDFFunc=parameters$getColDFFunc,
                             design=design,
                             transcriptMapperDF,
                             ignoreAfterBar=ignoreAfterBar,
                             txOut=parameters$txOut)
  
    line 313
    prefilter
    
    
    ddsCountsDF <- createComparableCounts( dds, filterRowCount=aedwip, filterSampleCount )
  ddsCountsDF <- createComparableCounts( dds, filterRowCount=aedwip, filterSampleCount )
  
  
  ret = list(ddsCountsDF,  dds, fileList) 
  names(ret) <- c("ddsCountsDF", "dds", "fileList")
  
  return(ret)
 }
```

## runNomalization -> loadAndNormlize -> getDESeqDataSet
```
getDESeqDataSet <- function(rootDir,
                            indexFilePath,
                            findFilesFunc,
                            getColDFFunc,
                            design,
                            transcriptMapperDF,
                            ignoreAfterBar=TRUE,
                            txOut=FALSE) {
    
  # tximport returns a list with matrices, "abundance", "counts", and "length", 
  # where the transcript level information is summarized to the gene-level. 
  # Typically, abundance is provided by the quantification tools as 
  # TPM (transcripts-per-million), while the counts are estimated counts 
  # (possibly fractional), and the "length" matrix contains the effective gene lengths
    
    # abundance == double, TPM column in salmon quant.sf
    # counts    == double, numReads Column in salmon quant.sf
    # length    == intetger , effective length column in salmon quant.sf
    txi <- tximport(allSalmonQuantFiles, 
                  type="salmon", 
                  tx2gene=transcriptMapperDF, 
                  ignoreAfterBar=ignoreAfterBar,
                  txOut=txOut)
    
    # get the meta data for each column of count data.
    colDF <- getColDFFunc(fileList)
    
    dds <- DESeqDataSetFromTximport(txi, colData=colDF, design)
  
  retList = List(dds, fileList)
  names(retList) <- c("dds", "fileList")
  
  return(retList) 
}
    
```

## runNomalization -> loadAndNormalize -> getDESeqDataSet -> tximport
https://github.com/mikelove/tximport/blob/master/R/tximport.R

```
line 361: salmon processing
txIdCol <- name
abundanceCol <- Tpm
countCol <- numReads
lengthCol <- effectiveLength

package readr is present
line 368
col.types <- readr::cols(
        readr::col_character(),readr::col_integer(),readr::col_double(),readr::col_double(),readr::col_double()

line 479, loop over all files
  for (i in seq_along(files)) {
    ...
    
      abundanceMatTx[,i] <- raw[[abundanceCol]]
      countsMatTx[,i] <- raw[[countsCol]]
      lengthMatTx[,i] <- raw[[lengthCol]]

      ...
line 547
    }
  ### --- END --- loop over files ###
  
  line 583
      txi <- list(abundance=abundanceMatTx,
                counts=countsMatTx,
                length=lengthMatTx,
                countsFromAbundance=countsFromAbundance)

line 643
txi[["countsFromAbundance"]] <- NULL
  txiGene <- summarizeToGene(txi, tx2gene, varReduce, ignoreTxVersion, ignoreAfterBar, countsFromAbundance)
  return(txiGene)
```

##  runNomalization -> loadAndNormalize ->  getDESeqDataSet -> tximport -> summarizeToGene
https://github.com/mikelove/tximport/blob/master/R/summarizeToGene.R

```
object argument is the txi list create on line 583 in tximport
line 32
txId <- rownames(abundanceMatTx)
stopifnot(all(txId == rownames(countsMatTx)))
stopifnot(all(txId == rownames(lengthMatTx)))
  
line 47
cleanTx2Gene removes duplicates and converts columns to factors
tx2gene <- cleanTx2Gene(tx2gene)    


lot of code to remove transcripts and genes not in the rownames of the matrices
then create a geneId vector which aligns to the matrices
```

### group by gene and count

```
runsum(x, group, reoder=TRUE)

group	
a vector or factor giving the grouping, with one element per row of x. Missing values will be treated as another group and a warning will be given.

reorder	
if TRUE, then the result will be in order of sort(unique(group)), if FALSE, it will be in the order that groups were encountered.

  countsMat <- rowsum(countsMatTx, geneId)
  
line  135
# countsFromAbundance = 'no'
out <- list(abundance=abundanceMat,
                counts=countsMat,
                length=lengthMat,
                countsFromAbundance=countsFromAbundance)
                
line 141

```


## state after tximport
- we have a list of 3 matrices. -
- They have been "grouped by gene" and counted
- <span style="color:red"> counts have not been converted to integer</span>

## runNomalization -> loadAndNormalize -> getDESeqDataSet -> DESeqDataSetFromTximport
DESeqUtils.R
-https://github.com/mikelove/DESeq2/blob/master/R/AllClasses.R
<span style="color:red"> # convert counts to integers </span>

```
line 408

  # convert counts to integers
  counts <- round(txi$counts)
  
  # help 'mode {base}'
  # Get or set the type or storage mode of an object.
  # this is not the 'mode' measure of central tendancy
  # R does not have a standard in-built function to calculate mode. 
  mode(counts) <- "integer"

  object <- DESeqDataSetFromMatrix(countData=counts, colData=colData, design=design, ...)

```

## runNomalization -> loadAndNormalize -> getDESeqDataSet -> DESeqDataSetFromTximport -> DESeqDataSetFromMatrix
https://github.com/mikelove/DESeq2/blob/master/R/AllClasses.R

<span style="color:red">our DESeq data set only as count data </span>


```
line 334

# santity check
# check that these agree in number
  stopifnot(ncol(countData) == nrow(colData))
  
  # we expect a matrix of counts, which are non-negative integers
  countData <- as.matrix( countData )
  
  # check if the rownames of colData are simply in different order
  # than the colnames of the countData, if so throw an error
  # as the user probably should investigate what's wrong
  
  
    se <- SummarizedExperiment(assays = SimpleList(counts=countData), colData = colData, ...)
    
    # DESeqDataSet does not do any numeric processing. Design is simply stored in returned object
    # it enforces non-negative count values
  object <- DESeqDataSet(se, design = design, ignoreRank)

  return(object)
  
  
 DESeqDataSet()
more sanity check
# before validity check, try to convert assay to integer mode
  if (any(is.na(assay(se))))
    stop("NA values are not allowed in the count matrix")
  if (any(assay(se) < 0)) {
    stop("some values in assay are negative")
  }
  if (!is.integer(assay(se))) {
    if (!is.numeric(assay(se))) {
      stop(paste("counts matrix should be numeric, currently it has mode:", mode(assay(se))))
    }
    if (any(round(assay(se)) != assay(se))) {
      stop("some values in assay are not integers")
    }
  
```



## runNomalization -> loadAndNormalize  -> createComparableCounts

DESeqUtils.R

```
line 11

# prefilter counts 
# filter rows:
# remove rows if they do not have a count of 5 or more in
# 4 or more samples
# dds is a DESeqDataSet object
keep <- rowSums(counts(dds) >= filterRowCount) >= filterSampleCount
table(keep)
dds <- dds[keep,]  


# https://www.rdocumentation.org/packages/DESeq2/versions/1.12.3/topics/estimateSizeFactors
  # This function estimates the size factors using the "median ratio method"
  # described by Equation 5 in Anders and Huber (2010). The estimated size
  # factors can be accessed using the accessor function sizeFactors
  dds <- estimateSizeFactors( dds )
  
  aedwip
```

## runNomalization -> loadAndNormalize  -> createComparableCounts -> estimateSizeFactors

line 367 https://github.com/mikelove/DESeq2/blob/master/R/methods.R

```

estimateSizeFactors.DESeqDataSet <- function(object, # object arg is a DESeqDataSet 
                                            type=c("ratio","poscounts","iterate"),
                                             locfunc=stats::median,
                                             geoMeans, 
                                             controlGenes, 
                                             normMatrix, 
                                             quiet=FALSE) {



line 397
else if (missing(normMatrix)) {
      sizeFactors(object) <- estimateSizeFactorsForMatrix(counts(object), locfunc=locfunc,
                                                          geoMeans=geoMeans,
                                                          controlGenes=controlGenes)
    } else {
      normalizationFactors(object) <- estimateNormFactors(counts(object),
                                                          normMatrix=normMatrix,
                                                          locfunc=locfunc,
                                                          geoMeans=geoMeans,
                                                          controlGenes=controlGenes)
                                                          

```

<span style="color:red">it is hard to find code, used print(funName) in RStudio console. It is very hard to follow code. I think this is the jest </span>

```
> print( estimateSizeFactorsForMatrix)
function (counts, locfunc = stats::median, geoMeans, controlGenes, 
    type = c("ratio", "poscounts")) 
{

    if (type == "ratio") {
            loggeomeans <- rowMeans(log(counts))
        }
        
   sf <- if (missing(controlGenes)) {
        apply(counts, 2, function(cnts) {
            exp(locfunc((log(cnts) - loggeomeans)[is.finite(loggeomeans) & 
                cnts > 0]))
        })
    }
    
    
  return sf
```

# <span style="color:red"> runNomalization -> runDifferentialExpression(dds, parameters) </span>

```
  ddsDESeq <- DESeq(dds)

```

# <span style="color:red"> runNomalization -> runDifferentialExpression(dds, parameters) -> DESeq() </span>
core.R

useful test function makeExampleDESeqDataSet
    
```
# general outline of the internal function calls.
# note: not all of these functions are exported.
#
# DESeq
# |- estimateSizeFactors
#    |- estimateSizeFactorsForMatrix
# |- estimateDispersions
#    |- estimateDispersionsGeneEst
#       |- fitNbinomGLMs
#          |- fitBeta (C++)
#       |- fitDisp (C++)
#    |- estimateDispersionsFit
#    |- estimateDispersionsMAP
#       |- estimateDispersionPriorVar
#       |- fitDisp (C++)
# |- nbinomWaldTest
#    |- fitGLMsWithPrior
#       |- fitNbinomGLMs
#          |- fitBeta (C++)
#       |- estimateBetaPriorVar
#       |- fitNbinomGLMs
#          |- fitBeta (C++)
#
```

line 279:
```
DESeq <- function(object, # object is a DESeqDataSet
                   test=c("Wald","LRT"),
                  fitType=c("parametric","local","mean", "glmGamPoi"),
                  sfType=c("ratio","poscounts","iterate"),
                  betaPrior,
                  full=design(object), 
                  reduced, 
                  quiet=FALSE,
                  minReplicatesForReplace=7, 
                  modelMatrixType,
                  useT=FALSE, 
                  minmu=if (fitType=="glmGamPoi") 1e-6 else 0.5,
                  parallel=FALSE, 
                  BPPARAM=bpparam()) {
                  
    arguments values
    test <- "Wald"
    fitType <- parametric
    stype <- ratio
    betaPrior ???
    full <- design
    reduce ????
    minReplicatesForReplace <- 7
    modelMatixType ???
    useT <- False
    minmu <- 0.5
    
    code begins
    
    dispersionEstimator <- "DESeq2"
    betaPrior <- FALSE
    
     # get rid of any NA in the mcols(mcols(object))
  object <- sanitizeRowRanges(object)
  
  line 346
  
  ilne 376
  
  line 392
      object <- estimateDispersions(object, fitType=fitType, quiet=quiet, modelMatrix=modelMatrix, minmu=minmu)

    
}

sanitizeRowRanges <- function(object) {
  if (is.null(mcols(mcols(object)))) {
    mcols(mcols(object)) <- DataFrame(type=rep("input",ncol(mcols(object))),
                                      description=character(ncol(mcols(object))))
  }
  class(mcols(mcols(object))$type) <- "character"
  class(mcols(mcols(object))$description) <- "character"
  mcols(mcols(object))$type[ is.na(mcols(mcols(object))$type) ] <- ""
  mcols(mcols(object))$description[ is.na(mcols(mcols(object))$description) ] <- ""
  object
}
 

methods.R line 504
estimateDispersions.DESeqDataSet <- function(object, fitType=c("parametric","local","mean", "glmGamPoi"),
                                             maxit=100, useCR=TRUE,
                                             weightThreshold=1e-2,
                                             quiet=FALSE, modelMatrix=NULL,
                                             minmu=if (fitType=="glmGamPoi") 1e-6 else 0.5) {
```
